In [2]:
import SimpleITK as sitk
import os
from matplotlib import pyplot as plt
import numpy as np

In [3]:
# LABELS : creation of a list of vectors --> manual classification
# IMG 53 
y_g1_53 = np.zeros(512)
y_g1_53[172:214] = 1
y_g1_53[311:347] = 1

# IMG 54
y_g1_54 = np.zeros(512)
y_g1_54[160:203] = 1
y_g1_54[290:326] = 1

# IMG 55 
y_g1_55 = np.zeros(512)
y_g1_55[180:226] = 1
y_g1_55[291:332] = 1

# list of vectors (one per image to provide to the function)
label = [y_g1_53, y_g1_54, y_g1_55]

In [4]:
def image_to_slices (image):
    
    '''
    the function takes a 3D image and converts it to sagital slices 
    It returns a list of slices
    
    '''
    shape = image.GetSize()
    image_array = sitk.GetArrayFromImage(image)   
    image_slices = []
    for i in range (shape[0]):
        sl_array = image_array[:,:,i]
        # from image to array reverse the image up down 
        sl_array = np.flipud(sl_array)
        image_slices.append(sl_array)
    
    return image_slices


In [5]:
def get_slices_from_paths(im_list):
    """
    function to convert image paths list of numpy images of slices
    :param im_list: list of paths to .nii.gz images
    :return: slice_list: list of 2D numpy arrays containing image slices
    """
    slice_list = []
    # each image has to be converted in sagital slice
    for j in range(len(im_list)):
        # read image
        image = sitk.ReadImage(im_list[j], sitk.sitkFloat32, imageIO="NiftiImageIO")
        # create a list of slices
        slice_list += image_to_slices(image)

    return slice_list

In [6]:
def preprocess_images(image, input_size):
    """
    Function to preprocess slices of image.
    :param image: list of 2D numpy arrays that contain saggital slices of hip/femur as numpy array
    :param input_size: input shape of image to reshape
    :return:
    """
    image_preprocessed = [np.reshape(slice, input_size) for slice in image]
    image_preprocessed = [(slice - np.min(slice)) / (np.max(slice) - np.min(slice)) for slice in image_preprocessed]
    return image_preprocessed

In [13]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras import applications, Model

def GetLeNet(n_base=32, input_shape=(128, 128)):
    """Definition of leNet tensor flow network architecture
    """
    # Building the following LeNet architecture step by step
    model = Sequential()
    # First layer: 32 2D convolutional with size of 3by3 with "rectified linear unit" as activation function.
    model.add(Conv2D(n_base, kernel_size=(3, 3), activation='relu',  strides=1, padding='same', input_shape=input_shape))
    # Second layer: 2D layer: pooling layer (max-pooling) with the size of 2by2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # 3rd layer: 64 2D convolutional with size of 3by3 with "rectified linear unit" as activation function.
    model.add(Conv2D(2*n_base, (3, 3), activation='relu', strides=1, padding='same'))
    # 4th layer: pooling layer (max-pooling) with the size of 2by2
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Vectorize the resulted image
    model.add(Flatten())
    # 5th layer: fully-connected layer (Dense) with the 128 nodes and "rectified linear unit" as activation function.
    model.add(Dense(4*n_base, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

In [14]:
def train_classifier(im_list, labels_list):
    """

    :param im_list: list of numpy 2D arrays slices of ct images
    :param labels_list: list of binary label whether or not images contain structure of interest
    :return: model: trained classifier that detects if structure is present
             shape: shape of input
    """


    '''
    train classifier
    NB --> sagital slices of different images have different size in y direction
    '''
    shape  = im_list[0].shape()
    images_preprocessed = preprocess_images(im_list, shape)
    model = GetLeNet(n_base=32, input_shape=shape)
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0),
              metrics=['accuracy'])

    return model, shape



In [9]:
atlas_images_base = os.path.join("data", "GROUP_images")
files_images = os.listdir(atlas_images_base)
atlas_images = [os.path.join(atlas_images_base, fil) for fil in files_images if ".nii" in fil]
atlas_images.sort()
slice_list = train_classifier(atlas_images, label)

AttributeError: 'str' object has no attribute 'shape'

In [ ]:
print(len(slice_list))
sitk.Show(slice_list[1204]) # slice 181 img 55  simple itk


In [ ]:



'''
# load group image 
im_base_dir = os.path.join("data", "GROUP_images")
g1_53_image = sitk.ReadImage(os.path.join(im_base_dir, "g1_53_image.nii.gz"), sitk.sitkFloat32, imageIO="NiftiImageIO")
g1_54_image = sitk.ReadImage(os.path.join(im_base_dir, "g1_54_image.nii.gz"), sitk.sitkFloat32, imageIO="NiftiImageIO")
g1_55_image = sitk.ReadImage(os.path.join(im_base_dir, "g1_55_image.nii.gz"), sitk.sitkFloat32, imageIO="NiftiImageIO")

# different size!!!!
print(g1_55_image.GetSize())
print(g1_54_image.GetSize())
print(g1_53_image.GetSize())

g1_53_slices = image_to_slices(g1_53_image)
g1_54_slices = image_to_slices(g1_54_image)
g1_55_slices = image_to_slices(g1_55_image)


#slices = g1_53_slices + g1_54_slices + g1_55_slices

'''